In [1]:
import cv2
import numpy as np

In [2]:
from tensorflow import keras

In [3]:
from keras_self_attention import SeqSelfAttention

In [4]:
model =  keras.models.load_model('lracn__Loss_0.594653308391571__Accuracy_0.8135592937469482.h5', 
                   custom_objects={'SeqSelfAttention': SeqSelfAttention})

In [5]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
CLASSES_LIST = ["jump", "punch", "pushup", "pullup", "handstand"]

In [6]:
def predict(videopath, SEQUENCE_LENGTH):
    video_reader = cv2.VideoCapture(videopath)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    frames_list = []
    
    predicted_class_name = ''

    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)
 

    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read() 
        if not success:
            break

        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255

        frames_list.append(normalized_frame)
    
    predicted_labels_probabilities = model.predict(np.expand_dims(frames_list, axis = 0))[0]
    predicted_label = np.argmax(predicted_labels_probabilities)
    predicted_class_name = CLASSES_LIST[predicted_label]
    
    print(f'Predicted Class: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')

    video_reader.release()

In [7]:
predict('hmdb51/punch/AdvancedBoxingTechniquesandExercises_punch_u_nm_np1_ri_med_24.avi',30)

Predicted Class: punch
Confidence: 0.995488703250885


In [8]:
predict('hmdb51/jump/THE_PROTECTOR_jump_f_nm_np1_ba_bad_90.avi',30)

Predicted Class: jump
Confidence: 0.9076400399208069


In [9]:
predict('hmdb51/z_testing/Aman_1.mp4',30)

Predicted Class: punch
Confidence: 0.9693729281425476


In [10]:
predict('hmdb51/z_testing/VID_20220512_162404.mp4', 30)

Predicted Class: pushup
Confidence: 0.5832154154777527


In [11]:
predict('hmdb51/z_testing/VID_20220512_162016.mp4', 30)

Predicted Class: pullup
Confidence: 0.4325949251651764


In [12]:
predict('hmdb51/z_testing/VID_20220512_161605.mp4', 30)

Predicted Class: handstand
Confidence: 0.9843642115592957


In [13]:
predict('hmdb51/z_testing/Jump5.mp4', 30)

Predicted Class: jump
Confidence: 0.9701799154281616
